<a href="https://colab.research.google.com/github/pojo-25/drugProject/blob/main/Deneme1ClasssicialModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import sys
sys.path.insert(0, os.getcwd()) # add current working directory to pythonpath

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
import pickle
import warnings
import argparse

In [112]:
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, confusion_matrix

In [1]:
url = 'https://anaconda.org/rdkit/rdkit/2018.09.1.0/download/linux-64/rdkit-2018.09.1.0-py36h71b666b_1.tar.bz2'
!curl -L $url | tar xj lib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3773    0  3773    0     0   8440      0 --:--:-- --:--:-- --:--:--  8421
100 20.2M  100 20.2M    0     0  5469k      0  0:00:03  0:00:03 --:--:-- 7646k


In [3]:
# move to python packages directory
!mv lib/python3.6/site-packages/rdkit /usr/local/lib/python3.6/dist-packages/
x86 = '/usr/lib/x86_64-linux-gnu'

In [4]:
!mv lib/*.so.* $x86/

In [5]:
# rdkit need libboost_python3.so.1.65.1
!ln -s $x86/libboost_python3-py36.so.1.65.1 $x86/libboost_python3.so.1.65.1

In [7]:
!pip install kora -q
import kora.install.rdkit

     |████████████████████████████████| 61kB 4.7MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 


In [9]:
from sklearn.feature_selection import chi2
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors, AllChem, ChemicalFeatures, Descriptors, Crippen, Lipinski
from rdkit.Chem.Pharm2D import Generate, Gobbi_Pharm2D
from rdkit.Chem.Pharm2D.SigFactory import SigFactory


In [10]:
fdefName = '/content/models/MinimalFeatures.fdef'
featFactory = ChemicalFeatures.BuildFeatureFactory(fdefName)

In [11]:
def extract_properties(column, include_3D=False):
    """Extract various 1D descriptors
    https://www.rdkit.org/docs/GettingStartedInPython.html#list-of-available-descriptors
    
    :param column: Pandas Series, containing smiles or RDKit mol object
    :param from_smiles: bool, indicate whether column contains smiles string
    :return: feature_properties: Pandas Series, containing 1D descriptors
    """

    def extract(x, from_smiles):
        if from_smiles:
            mol = Chem.MolFromSmiles(x)
        else:
            mol = x
        
        if (mol is None) or (len(mol.GetAtoms()) == 0):
            if include_3D:
                return [0] * 29
            else:
                return [0] * 24
        else:
            logP = Crippen.MolLogP(mol)
            refractivity = Crippen.MolMR(mol)
            
            weight = Descriptors.MolWt(mol)
            exact_weight = Descriptors.ExactMolWt(mol)
            heavy_weight = Descriptors.HeavyAtomMolWt(mol)
            heavy_count = Lipinski.HeavyAtomCount(mol)
            nhoh_count = Lipinski.NHOHCount(mol)
            no_count = Lipinski.NOCount(mol)
            hacceptor_count = Lipinski.NumHAcceptors(mol)
            hdonor_count = Lipinski.NumHDonors(mol)
            hetero_count = Lipinski.NumHeteroatoms(mol)
            rotatable_bond_count = Lipinski.NumRotatableBonds(mol)
            valance_electron_count = Descriptors.NumValenceElectrons(mol)
            amide_bond_count = rdMolDescriptors.CalcNumAmideBonds(mol)
            aliphatic_ring_count = Lipinski.NumAliphaticRings(mol)
            aromatic_ring_count = Lipinski.NumAromaticRings(mol)
            saturated_ring_count = Lipinski.NumSaturatedRings(mol)
            aliphatic_cycle_count = Lipinski.NumAliphaticCarbocycles(mol)
            aliphaticHetero_cycle_count = Lipinski.NumAliphaticHeterocycles(mol)
            aromatic_cycle_count = Lipinski.NumAromaticCarbocycles(mol)
            aromaticHetero_cycle_count = Lipinski.NumAromaticHeterocycles(mol)
            saturated_cycle_count = Lipinski.NumSaturatedCarbocycles(mol)
            saturatedHetero_cycle_count = Lipinski.NumSaturatedHeterocycles(mol)
            
            tpsa = rdMolDescriptors.CalcTPSA(mol)
            
            if include_3D:
                mol_3D=Chem.AddHs(mol)
                AllChem.EmbedMolecule(mol_3D)
                AllChem.MMFFOptimizeMolecule(mol_3D)
                eccentricity = rdMolDescriptors.CalcEccentricity(mol_3D)
                asphericity = rdMolDescriptors.CalcAsphericity(mol_3D)
                spherocity = rdMolDescriptors.CalcSpherocityIndex(mol_3D)
                inertial = rdMolDescriptors.CalcInertialShapeFactor(mol_3D)
                gyration = rdMolDescriptors.CalcRadiusOfGyration(mol_3D)
            
                return [logP, refractivity, weight, exact_weight, heavy_weight, heavy_count, nhoh_count, no_count,
                        hacceptor_count, hdonor_count, hetero_count, rotatable_bond_count, valance_electron_count,
                        amide_bond_count, aliphatic_ring_count, aromatic_ring_count, saturated_ring_count,
                        aliphatic_cycle_count, aliphaticHetero_cycle_count, aromatic_cycle_count,
                        aromaticHetero_cycle_count, saturated_cycle_count, saturatedHetero_cycle_count, tpsa,
                        eccentricity, asphericity, spherocity, inertial, gyration]
            else:
                return [logP, refractivity, weight, exact_weight, heavy_weight, heavy_count, nhoh_count, no_count,
                        hacceptor_count, hdonor_count, hetero_count, rotatable_bond_count, valance_electron_count,
                        amide_bond_count, aliphatic_ring_count, aromatic_ring_count, saturated_ring_count,
                        aliphatic_cycle_count, aliphaticHetero_cycle_count, aromatic_cycle_count,
                        aromaticHetero_cycle_count, saturated_cycle_count, saturatedHetero_cycle_count, tpsa]
                
    feature_properties = column.apply(lambda x: extract(x))

    return np.array(list(feature_properties))

In [89]:
####################
## 2D descriptors ##
####################
def extract_MQNs(column, from_smiles=True):
    """Extract MQN features from smiles
    :param column: Pandas Series, containing smiles or RDKit mol object
    :param from_smiles: bool, indicate whether column contains smiles string
    :return feature_MQN: Pandas Series, containing 42 MQN feature
    """
    
    def get_MQNs(x, from_smiles):
        if from_smiles:
            mol = Chem.MolFromSmiles(x)
        else:
            mol = x
        if (mol is None) or (len(mol.GetAtoms()) == 0):
            return [0]*42
        else:
           return rdMolDescriptors.MQNs_(mol) 
       
    feature_MQN = column.apply(lambda x: get_MQNs(x, from_smiles))
    return np.array(list(feature_MQN))


def extract_Morganfp(column, radius=2, nBits=2048, useFeatures=False, from_smiles=True):
    """Extract Morganfingerprint
    :param column: Pandas Series, containing smiles or RDKit mol object
    :param radius: int, indicates the radius in the Morgan fingerprint calculation.
    :param nBits: int, the number of bits in the resulting bit vector.
    :param useFeatures: bool, whether atoms' specific features are used
    :param from_smiles: bool, indicate whether column contains smiles string
    :return: feature_morgan: Pandas Series, containing Morganfingerprint features
    """
    
    def get_Morganfp(x, from_smiles):
        if from_smiles:
            mol = Chem.MolFromSmiles(x)
        else:
            mol = x
        if (mol is None) or (len(mol.GetAtoms()) == 0):
            return [0]*nBits
        else:
           return AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits, useFeatures=useFeatures) 
        
    feature_morgan = column.apply(lambda x: get_Morganfp(x, from_smiles))
    return np.array(list(feature_morgan))


def extract_Pharm2D(column, minPointCount=2, maxPointCount=3, bins=[(0,2),(2,5),(5,8)], from_smiles=True):
    """Extract Pharm2D fingerprint
    :param column: Pandas Series, containing smiles or RDKit mol object
    :param minPointCount: int
    :param maxPointCount: int
    :param bins: lits of tuples
    :param from_smiles: bool, indicate whether column contains smiles string
    :return: feature_Pharm2D: Pandas Series, containing Pharm2D features
    """
    sigFactory = SigFactory(featFactory,
                            minPointCount=minPointCount,
                            maxPointCount=maxPointCount,
                            trianglePruneBins=False)
    sigFactory.SetBins(bins)
    sigFactory.Init()
    
    def get_Pharm2D(x):
        mol = Chem.MolFromSmiles(x)
        if (mol is None) or (len(mol.GetAtoms()) == 0):
            return [0]*sigFactory.GetSigSize()
        else:
           return Generate.Gen2DFingerprint(mol, sigFactory)     

    fp = column.apply(lambda x: get_Pharm2D(x))
    return np.array(list(fp))


def extract_Gobbi_Pharm2D(column, from_smiles=True):
    """Extract Gobbi Pharm2D fingerprint
    :param column: Pandas Series, containing smiles or RDKit mol object
    :param from_smiles: bool, indicate whether column contains smiles string
    :return: feature_Gobbi Pharm2D: Pandas Series, containing Gobbi Pharm2D  features
    """
    
    def get_Gobbi_Pharm2D(x, from_smiles):
        if from_smiles:
            mol = Chem.MolFromSmiles(x)
        else:
            mol = x
        return Generate.Gen2DFingerprint(mol, Gobbi_Pharm2D.factory)
        
    feature_Gobbi = column.apply(lambda x: get_Gobbi_Pharm2D(x, from_smiles))
    return np.array(list(feature_Gobbi))


####################
## 3D descriptors ##
####################
def extract_RDF(column, from_smiles=True):
    """Extract RDF descriptor
    :param column: Pandas Series, containing smiles or RDKit mol object
    :param from_smiles: bool, indicate whether column contains smiles string
    :return: feature_RDF: Pandas Series, containing 210 RDF features
    """
    def get_RDF(x, from_smiles):
        if from_smiles:
            mol = Chem.MolFromSmiles(x)
        else:
            mol = x
        if (mol is None) or (len(mol.GetAtoms()) == 0):
            return [0]*210
        else:
            mol_3D=Chem.AddHs(mol)
            AllChem.EmbedMolecule(mol_3D)
            AllChem.MMFFOptimizeMolecule(mol_3D)
            return rdMolDescriptors.CalcRDF(mol_3D) 
        
    feature_RDF = column.apply(lambda x: get_RDF(x, from_smiles))
    return np.array(list(feature_RDF))


def extract_AUTOCORR3D(column, from_smiles=True):
    """Extract AUTOCORR3D descriptor
    :param column: Pandas Series, containing smiles or RDKit mol object
    :param from_smiles: bool, indicate whether column contains smiles string
    :return: feature_AUTOCORR3D: Pandas Series, containing 80 AUTOCORR3D features
    """
    def get_AUTOCORR3D(x, from_smiles):
        if from_smiles:
            mol = Chem.MolFromSmiles(x)
        else:
            mol = x
        if (mol is None) or (len(mol.GetAtoms()) == 0):
            return [0]*80
        else:
            mol_3D=Chem.AddHs(mol)
            AllChem.EmbedMolecule(mol_3D)
            AllChem.MMFFOptimizeMolecule(mol_3D)
            return rdMolDescriptors.CalcAUTOCORR3D(mol_3D) 
        
    feature_AUTOCORR3D = column.apply(lambda x: get_AUTOCORR3D(x, from_smiles))
    return np.array(list(feature_AUTOCORR3D)), np.arange(80)


def extract_MORSE(column, from_smiles=True):
    """Extract MORSE descriptor
    :param column: Pandas Series, containing smiles or RDKit mol object
    :param from_smiles: bool, indicate whether column contains smiles string
    :return: feature_MORSE: Pandas Series, containing 224 MORSE features
    """
    def get_MORSE(x, from_smiles):
        if from_smiles:
            mol = Chem.MolFromSmiles(x)
        else:
            mol = x
        if (mol is None) or (len(mol.GetAtoms()) == 0):
            return [0]*224
        else:
            mol_3D=Chem.AddHs(mol)
            AllChem.EmbedMolecule(mol_3D)
            AllChem.MMFFOptimizeMolecule(mol_3D)
            return rdMolDescriptors.CalcMORSE(mol_3D) 
        
    feature = column.apply(lambda x: get_MORSE(x, from_smiles))
    return np.array(list(feature)), np.arange(224)


def extract_WHIM(column, from_smiles=True):
    """Extract WHIM descriptor
    :param column: Pandas Series, containing smiles or RDKit mol object
    :param from_smiles: bool, indicate whether column contains smiles string
    :return: feature_WHIM: Pandas Series, containing 114 WHIM features
    """
    def get_WHIM(x, from_smiles):
        if from_smiles:
            mol = Chem.MolFromSmiles(x)
        else:
            mol = x
        if (mol is None) or (len(mol.GetAtoms()) == 0):
            return [0]*114
        else:
            mol_3D=Chem.AddHs(mol)
            AllChem.EmbedMolecule(mol_3D)
            AllChem.MMFFOptimizeMolecule(mol_3D)
            return rdMolDescriptors.CalcWHIM(mol_3D) 
        
    feature = column.apply(lambda x: get_WHIM(x, from_smiles))
    return np.array(list(feature)), np.arange(114)


def extract_GETAWAY(column, from_smiles=True):
    """Extract GETAWAY descriptor. GETAWAT descriptors have NaN values sometimes.
    :param column: Pandas Series, containing smiles or RDKit mol object
    :param from_smiles: bool, indicate whether column contains smiles string
    :return: feature_GETAWAY: Pandas Series, containing 273 GETAWAY features
    """
    def get_GETAWAY(x):
        mol = Chem.MolFromSmiles(x)
        if (mol is None) or (len(mol.GetAtoms()) == 0):
            return [0]*273
        else:
            mol_3D=Chem.AddHs(mol)
            AllChem.EmbedMolecule(mol_3D)
            AllChem.MMFFOptimizeMolecule(mol_3D)
            return rdMolDescriptors.CalcGETAWAY(mol_3D) 
        
    feature = column.apply(lambda x: get_GETAWAY(x))
    return np.array(list(feature)), np.arange(273)

In [90]:
def extract_features(column, method=['morgan'], from_smiles=True):
    """Extract 1D, 2D and 3D descriptors
    :param column: Pandas Series, containing smiles or RDKit mol object
    :param method: list, containing names of descriptors to extract
    :param from_smiles: bool, indicate whether column contains smiles string
    :return features: list of features extracted with method list
    """

    feature_list = []
    if 'morgan' in method:
        feature_list.append(extract_Morganfp(column))
    if 'mqn' in method:
        feature_list.append(extract_MQNs(column))
    if 'pharm2D' in method:
        feature_list.append(extract_Pharm2D(column))
    if 'gobbi' in method:
        feature_list.append(extract_Gobbi_Pharm2D(column))
    if 'physical' in method:
        feature_list.append(extract_properties(column, include_3D=False))
    if 'physical3D' in method:
        feature_list.append(extract_properties(column, include_3D=True))
    if 'autocorr3D' in method:
        feature_list.append(extract_AUTOCORR3D(column))
    if 'rdf' in method:
        feature_list.append(extract_RDF(column))
    if 'morse' in method:
        feature_list.append(extract_MORSE(column))
    if 'whim' in method:
        feature_list.append(extract_WHIM(column))
    if 'getaway' in method:
        feature_list.append(extract_GETAWAY(column))

    return np.concatenate(feature_list, axis=1)
        
     
def filter_feature_Chi2(feature_column, target_column, threshold=None):
    """Filter feature using Chi2 test
    :param: feature_column: numpy array containing feature
    :param: target_column: numpy array containing target variable
    :param: threshold: threshold to filter using Chi2.
                        threshold=None means all features with non Nan Chi2 pval will be returned.
    :return: feature_selected: features that are signficant
    :return: pval_significant: list of bool to indicate which features are significant
    """

    # Perform Chi2 test
    chi2_stats, pval = chi2(feature_column, target_column)

    # select only significant pvals
    pval_result = pval.copy()
    if threshold:
        pval_result[np.isnan(pval_result)] = 100  # replace Nan with any large value
        pval_significant = pval_result <= threshold
    else:
        pval_significant = np.logical_not(np.isnan(pval))

    # select features with significant pvals
    feature_selected = feature_column[:, pval_significant]

    return feature_selected, pval_significant

In [91]:
col_smiles = 'smiles'
col_target = 'HIV_active'

METRIC_ACCURACY = 'accuracy'
METRIC_F1_SCORE = 'f1-score'
METRIC_COHEN_KAPPA = 'Cohen kappa'
METRIC_CONFUSION_MATRIX = 'Confusion Matrix'


CLASSES = ['benign', 'malignant']
TEST_RATIO = 0.2
SEED = 0

data_path = '/content/data/HIV.csv'

In [92]:
def read_data(data_path, col_smiles='smiles', col_target='HIV_active'):
    """Split original data into train data and test data.
    :param data_path: str, path to the a CSV data file
    :param col_smiles: str, name of smiles column
    :param col_target: str, name of target column
    :param test_ratio: float, proportion of the original data for testset, must be from 0 to 1
    :param seed: int, randomization seed for reproducibility
    :return (X, y)
    """
    

    # read data
    df = pd.read_csv(data_path, sep=',')
    df_no_na = df[[col_smiles, col_target]].dropna()

    X = df_no_na[col_smiles]
    y = df_no_na[col_target].values
    
    return X, y

                
def get_prediction_score(y_label, y_predict):
    """Evaluate predictions using different evaluation metrics.
    :param y_label: list, contains true label
    :param y_predict: list, contains predicted label
    :return scores: dict, evaluation metrics on the prediction
    """
    scores = {}
    scores[METRIC_ACCURACY] = accuracy_score(y_label, y_predict)
    scores[METRIC_F1_SCORE] = f1_score(y_label, y_predict, labels=None, average='macro', sample_weight=None)
    scores[METRIC_COHEN_KAPPA] = cohen_kappa_score(y_label, y_predict)
    scores[METRIC_CONFUSION_MATRIX] = confusion_matrix(y_label, y_predict)
    
    return scores

In [93]:
import os
import sys
sys.path.insert(0, os.getcwd()) # add current working directory to pythonpath

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
import pickle
import warnings
import argparse

In [94]:
def train_model(model, X_train, y_train, parameters, n_splits=3):
    """Train model with Grid-search cross validation to find the best hyperparameter
    :param model: Scikit-learn estimator
    :param X_train: trainset features
    :param y_train: trainset label
    :param parameters: dict, key is hyper parameter name and value is a list of hyper parameter values
    :return best_estimator: Scikit-learn estimator with the best hyper parameter
    :return best_score: best accuracy score
    :return best_param: dict, best hyper parameter
    """
    
    splits = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0).split(X=X_train, y=y_train)
    
    clf = GridSearchCV(model, parameters, cv=splits, scoring=make_scorer(accuracy_score))
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')  # disable the warning on default optimizer
        clf.fit(X_train, y_train)

    return clf.best_estimator_, clf.best_score_, clf.best_params_


def evaluate_model(model, X_train, y_train, X_test, y_test):
    """Evaluate model on testset
    :param model: Scikit-learn estimator
    :param X_train: trainset features
    :param y_train: trainset label
    :param X_test: testset features
    :param y_test: testset label
    :param parameters: dict, key is hyper parameter name and value is a list of hyper parameter values
    :return model: Scikit-learn estimator, fitted on the whole trainset
    :return y_predict: prediction on test set
    :return scores: dict, evaluation metrics on test set
    """
    
    # Refit the model on the whole train set
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')  # disable the warning on default optimizer
        model.fit(X_train, y_train)
        
    # Evaluate on test set
    y_predict = model.predict(X_test)
    scores = None
    if y_test is not None:
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')  # disable the warning on f1-score with not all labels
            scores = get_prediction_score(y_test, y_predict)

    return model, y_predict, scores


def build_base_models(X_train, y_train):
    """Train and evaluate different base models. "Base" means the model is not a stacking model. 
    :param X_train: trainset features
    :param y_train: trainset label
    :return fitted_models: list, contains fitted Scikit-learn estimators
    :return model_names: list, names of fitted Scikit-learn estimators
    :return model_scores: list, contains scores on test set for fitted Scikit-learn estimators.
                    Each score is a dict of evaluation metrics
    """
    
    ########################
    # DEFINE BASE MODELS ###
    ########################
    models = []
    model_params = []
    model_names = []
    
    # Random forest model
    for n_estimators in [500, 1000, 2000]:
        for max_depth in [3, 5, 7]:
            models.append(RandomForestClassifier(max_features='sqrt', class_weight='balanced', random_state=0))
            model_params.append({'n_estimators':[n_estimators], 'max_depth':[max_depth]})
            model_names.append('Random Forest')   
    
    # Boosted Tree
    for n_estimators in [500, 1000, 2000]:
        for max_depth in [3, 5, 7]:
            for learning_rate in [0.01, 0.1]:
                models.append(GradientBoostingClassifier(subsample=0.7, max_features='sqrt', random_state=0))
                model_params.append({'n_estimators':[n_estimators], 'max_depth':[max_depth], 'learning_rate':[learning_rate]})
                model_names.append('Gradient Boosting Machine')
    
    # SVM
#    for kernel in ['linear', 'rbf']:
#        for C in [1.0, 10.0, 100.0, 1000.0]:
#            models.append(SVC(probability=True, gamma='auto', tol=0.001, cache_size=200, class_weight='balanced',
#                              random_state=0,
#                              decision_function_shape='ovr'))
#            model_params.append({'kernel':[kernel], 'C':[C]})
#            model_names.append('Support Vector Machine')
    
    # Logistic regression model
    for penalty in ['l1', 'l2']:
        for C in [1.0, 10.0, 100.0, 1000.0]:
            models.append(linear_model.LogisticRegression(max_iter=500, solver='liblinear', multi_class='ovr',
                                                          class_weight='balanced', random_state=0))
            model_params.append({'penalty':[penalty], 'C':[C]})
            model_names.append('Logistic Regression')
        
    # KNN
#    for n_neighbors in [5, 10, 15]:
#        for weights in ['uniform', 'distance']:
#            models.append(KNeighborsClassifier())
#            model_params.append({'n_neighbors':[n_neighbors], 'weights':[weights]})
#            model_names.append('K Nearest Neighbour')
            
    ##################################
    # TRAIN AND EVALUATE BASE MODELS #
    ##################################
    fitted_models = []
    model_scores = []
    for i in range(len(models)):
        print('Evaluating model {} of {}: {}'.format((i+1), len(models), model_names[i]))
        model = models[i]
        fitted_cv, _, _ = train_model(model=model, X_train=X_train, y_train=y_train, parameters=model_params[i])
        fitted_whole_set, _, score = evaluate_model(model=fitted_cv, X_train=X_train, y_train=y_train,
                                                    X_test=X_test, y_test=y_test)
        fitted_models.append(fitted_whole_set)
        model_scores.append(score)
        print(model_names[i], score)
        
    return fitted_models, model_names, model_scores


def build_stack_models(base_models, X_train, y_train):
    """Train and evaluate different stack models
    :param base_models: list, contains fitted base models, which are Scikit-learn estimators
    :param X_train: trainset features
    :param y_train: trainset label
    :return stack_fitted_models: list, contains fitted Scikit-learn estimators
    :return stack_model_names: list, names of fitted Scikit-learn estimators
    :return stack_model_scores: list, contains scores on test set for fitted Scikit-learn estimators.
                    Each score is a dict of evaluation metrics
    """
    
    ###############################
    ### PREPARE DATA FOR STACKING #
    ###############################
    print('Preparing data for model stacking')    
    # Get base models' prediction for test set: simply use the trained models to predict on test set
    X_test_stack = np.zeros([X_test.shape[0], len(base_models)])
    for i in range(len(base_models)):
        model = base_models[i]
        X_test_stack[:, i] = model.predict(X_test)
            
    # Get base models' prediction for train set: use 3-fold split, train model on 2 parts and predict on 3rd part
    splits = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0).split(X=X_train, y=y_train)
    X_train_stack = np.zeros([X_train.shape[0], len(base_models)])
    for train_index, val_index in splits:
        # train and validation set
        X_tr, X_val = X_train[train_index], X_train[val_index]
        y_tr, _ = y_train[train_index], y_train[val_index]

        # Fit model
        for i in range(len(base_models)):
            model = base_models[i]
            with warnings.catch_warnings():
                warnings.simplefilter('ignore')  # disable the warning on default optimizer
                model.fit(X_tr, y_tr)
            X_train_stack[val_index, i] = model.predict(X_val)

    # Add base models' predictions into the feature space
    X_train_stack = np.concatenate([X_train, X_train_stack], axis=-1)
    X_test_stack = np.concatenate([X_test, X_test_stack], axis=-1)
          
    ########################
    # DEFINE STACK MODELS ##
    ########################
    stack_models = []
    stack_model_names = []
    stack_model_params = []
    
    stack_models.append(linear_model.LogisticRegression(max_iter=500, solver='liblinear', multi_class='ovr',
                                                        class_weight='balanced', random_state=0))
    stack_model_names.append('Stack Logistic Regression')
    stack_model_params.append({'penalty':['l1', 'l2'], 'C':[1.0, 10.0, 100.0, 1000.0]})
    
    stack_models.append(RandomForestClassifier(class_weight='balanced', random_state=0))
    stack_model_names.append('Stack Random Forest')
    stack_model_params.append({'n_estimators':[500, 1000, 2000], 'max_depth':[3, 5, 7]})
    
    stack_models.append(GradientBoostingClassifier(subsample=0.7, max_features='sqrt', learning_rate=0.01,
                                                   random_state=0))
    stack_model_names.append('Stack Gradient Boosting Machine')
    stack_model_params.append({'n_estimators':[500, 1000, 2000], 'max_depth':[3, 5, 7]})
    
#    stack_models.append(SVC(probability=True, gamma='auto', tol=0.001, cache_size=200, random_state=0,
#                             decision_function_shape='ovr', class_weight='balanced'))
#    stack_model_names.append('Stack Support Vector Machine')
#    stack_model_params.append({'kernel':['linear', 'rbf'], 'C':[1.0, 10.0, 100.0, 1000.0]})
    
#    stack_models.append(KNeighborsClassifier())
#    stack_model_names.append('Stack K Nearest Neighbour')
#    stack_model_params.append({'n_neighbors':[5, 10, 15], 'weights':['uniform', 'distance']})          

    #########################
    # EVALUATE STACK MODELS #
    #########################
    stack_fitted_models = []
    stack_model_scores = []
    for i in range(len(stack_models)):
        print('Evaluating model {} of {}: {}'.format((i+1), len(stack_models), stack_model_names[i]))
        model = stack_models[i]
        fitted_cv, _, _ = train_model(model=model, X_train=X_train_stack, y_train=y_train,
                                      parameters=stack_model_params[i])
        fitted_whole_set, _, score = evaluate_model(model=fitted_cv, X_train=X_train_stack, y_train=y_train,
                                                    X_test=X_test_stack, y_test=y_test)
        stack_fitted_models.append(fitted_whole_set)
        stack_model_scores.append(score)
        print(stack_model_names[i], score)
        
    return stack_fitted_models, stack_model_names, stack_model_scores
        

In [95]:
col_smiles = 'smiles'
col_target = 'HIV_active'

METRIC_ACCURACY = 'accuracy'
METRIC_F1_SCORE = 'f1-score'
METRIC_COHEN_KAPPA = 'Cohen kappa'
METRIC_CONFUSION_MATRIX = 'Confusion Matrix'


CLASSES = ['benign', 'malignant']
TEST_RATIO = 0.2
SEED = 0

data_path = '/content/data/HIV.csv'

In [96]:
WORK_DIRECTORY = '/content/data'
DATA_FILE = 'HIV.csv'

data_path = os.path.join(WORK_DIRECTORY, DATA_FILE)
n_splits = 3
save_path = WORK_DIRECTORY


# Read data
X, y = read_data(data_path, col_smiles='smiles', col_target='HIV_active')


In [97]:
print(type(y))

<class 'numpy.ndarray'>


In [98]:
X_train = X.values
y_train_asArg = y


In [99]:
# Get train and test set
smiles_train, smiles_test, y_train, y_test = train_test_split(X_train, y_train_asArg, test_size = TEST_RATIO, shuffle=True, stratify=y, random_state=SEED)

In [100]:
print(y_test)

[0 0 0 ... 0 0 0]


In [101]:
print(type(smiles_train))
print((X_train))

<class 'numpy.ndarray'>
['CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)=[O+]2'
 'C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3)CC(c3ccccc3)=[O+]2)[O+]=C(c2ccccc2)C1'
 'CC(=O)N1c2ccccc2Sc2c1ccc1ccccc21' ...
 'Cc1ccc(N2C(=O)C3c4[nH]c5ccccc5c4C4CCC(C(C)(C)C)CC4C3C2=O)cc1'
 'Cc1cccc(N2C(=O)C3c4[nH]c5ccccc5c4C4CCC(C(C)(C)C)CC4C3C2=O)c1'
 'CCCCCC=C(c1cc(Cl)c(OC)c(-c2nc(C)no2)c1)c1cc(Cl)c(OC)c(-c2nc(C)no2)c1']


In [102]:
df = pd.DataFrame(smiles_train, columns = ['Column_A'])

In [103]:
print(type(df['Column_A']))

<class 'pandas.core.series.Series'>


In [104]:
# Extract features
feature_train = extract_features(df['Column_A'], method=['morgan', 'mqn'],
                                                    from_smiles=True)


In [105]:
df = pd.DataFrame(smiles_test, columns = ['Column_B'])

In [107]:
feature_test = extract_features(df['Column_B'], method=['morgan', 'mqn'],
                                                    from_smiles=True)

In [109]:
# Filter out irrelevant features
feature_train_filtered, pval_significant = filter_feature_Chi2(feature_train, y_train)
feature_test_filtered = feature_test[:, pval_significant]


In [110]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(feature_train_filtered)
X_test = scaler.transform(feature_test_filtered)


In [ ]:
# Build base models
base_models, base_model_names, base_model_scores = build_base_models(X_train, y_train)

In [ ]:
if save_path is not None:
      # Save base models    
      os.makedirs(os.path.join(save_path, 'base_models'), exist_ok=True)
      for i in range(len(base_models)):
          with open(os.path.join(save_path, 'base_models', 'base_model_' + str(i+1) + '.pkl'), 'wb') as f:
              pickle.dump(base_models[i], f)

In [ ]:
# Build level 1 stack models
stack_fitted_models, stack_model_names, stack_model_scores = build_stack_models(base_models.copy(),
                                                                                X_train, y_train)

In [116]:
if save_path is not None:
    # Save base models    
    os.makedirs(os.path.join(save_path, 'stacking_models'), exist_ok=True)
    for i in range(len(stack_fitted_models)):
        with open(os.path.join(save_path, 'stacking_models', stack_model_names[i] + '.pkl'), 'wb') as f:
            pickle.dump(stack_fitted_models[i], f)

NameError: ignored

In [ ]:
# Summarize model performance
model_df = pd.DataFrame({'model': base_model_names + stack_model_names,
                          METRIC_ACCURACY: [score[METRIC_ACCURACY] for score in base_model_scores]\
                              + [score[METRIC_ACCURACY] for score in stack_model_scores],
                        METRIC_F1_SCORE: [score[METRIC_F1_SCORE] for score in base_model_scores]\
                              + [score[METRIC_F1_SCORE] for score in stack_model_scores],
                        METRIC_COHEN_KAPPA: [score[METRIC_COHEN_KAPPA] for score in\
                                                    base_model_scores]\
                              + [score[METRIC_COHEN_KAPPA] for score in stack_model_scores],
                        METRIC_CONFUSION_MATRIX: [score[METRIC_CONFUSION_MATRIX] for score in\
                                                          base_model_scores]\
                              + [score[METRIC_CONFUSION_MATRIX] for score in stack_model_scores]                            
                          })
model_df = model_df[['model', METRIC_ACCURACY, METRIC_F1_SCORE, METRIC_COHEN_KAPPA,
                      METRIC_CONFUSION_MATRIX]]
model_df.to_csv(os.path.join(WORK_DIRECTORY, 'summary_stacking_model.csv'), index=False)
model_df.sort_values(by=[METRIC_ACCURACY, METRIC_F1_SCORE, METRIC_COHEN_KAPPA],
                      ascending=False, inplace=True)
print('Best model:\n' + str(model_df.iloc[0]))